In [17]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import warnings
warnings.simplefilter("ignore")

train_x = pd.read_csv("train_x.csv")
train_y = pd.read_csv("train_y.csv")["応募数 合計"]
test = pd.read_csv("test_x.csv")
# testファイルのお仕事No.を抽出
test_No = test["お仕事No."]

all = pd.concat([train_x, test]).reset_index(drop=True)
all = all.dropna(how='any', axis=1).drop('お仕事No.', axis=1)

print(train_x.shape, test.shape, all.shape)

(15853, 212) (3391, 212) (19244, 102)


In [18]:
pd.set_option("display.max_rows", 300)

all.head()

,職場の様子,勤務地固定,休日休暇(月曜日),大手企業,週1日からOK,交通費別途支給,残業月20時間以上,職種コード,1日7時間以下勤務OK,ミドル（40〜）活躍中,...,産休育休取得事例あり,扶養控除内,動画ファイル名,給与/交通費 給与下限,対象者設定 性別,WEB登録OK,オフィスが禁煙・分煙,勤務地 市区町村コード,勤務地 最寄駅1（駅名）,残業なし
0,3,1,0,0,0,0,0,22010,1,1,...,1,0,022_190725.mp4,1550,1,1,1,121,北千住,1
1,3,1,0,1,0,0,0,20020,1,1,...,1,0,022_190725.mp4,1800,1,1,1,101,新御茶ノ水,1
2,3,1,0,1,0,0,0,20020,1,1,...,1,0,022_190725.mp4,1800,1,1,1,101,新御茶ノ水,1
3,2,1,0,0,0,0,1,20020,0,1,...,1,0,022_190725.mp4,1650,1,1,1,102,八丁堀,0
4,2,1,0,0,0,0,1,20020,0,1,...,1,0,022_190725.mp4,1650,1,1,1,102,八丁堀,0


# 休暇曜日を平日と休日で分ける

In [19]:
# ----------------------------------------------------------------------------------------------------------------------------------------------
# 平日と休日にまとめる
# ----------------------------------------------------------------------------------------------------------------------------------------------
all["weekday"] = all["休日休暇(月曜日)"] + all["休日休暇(火曜日)"] + all["休日休暇(水曜日)"] + all["休日休暇(木曜日)"] + all["休日休暇(金曜日)"]
all["holiday"] = all["休日休暇(土曜日)"] + all["休日休暇(日曜日)"] + all["休日休暇(祝日)"]


# ----------------------------------------------------------------------------------------------------------------------------------------------
# 該当(休日情報)のデータの削除
# ----------------------------------------------------------------------------------------------------------------------------------------------
day = ["休日休暇(月曜日)", "休日休暇(火曜日)", "休日休暇(水曜日)", "休日休暇(木曜日)", "休日休暇(金曜日)", 
       "休日休暇(土曜日)", "休日休暇(日曜日)", "休日休暇(祝日)", "土日祝のみ勤務", "平日休みあり", "土日祝休み"]

all = all.drop(day, axis=1)

#土日祝のみ勤務
#平日休みあり

# 年齢に関するデータについて

In [20]:
# --------------------------------------------------------
# 年齢のデータに0が含まれるか確認
# --------------------------------------------------------
age = ["20代活躍中", "30代活躍中", "ミドル（40〜）活躍中"]

for i in age:
    if (all[i].loc[all[i] == 0].count() >= 1):
        print(i, "はどこかしらで0がある")
    else:
        print(i, "は全て1")


# --------------------------------------------------------
# 年齢のデータを削除
# --------------------------------------------------------
all = all.drop(age, axis=1)

20代活躍中 は全て1
30代活躍中 は全て1
ミドル（40〜）活躍中 は全て1


# 週の最低出勤数について

In [21]:
attendance = ["週1日からOK", "週2・3日OK", "週4日勤務"]
number_attendance = [1, 2.5, 4]

# -------------------------------------------------------------------------
# 最低出勤日数/週 を求める．リストattendanceが全て0である求人は5日/週とする
# -------------------------------------------------------------------------
all["attendance"] = 5.0
for i, j in zip (number_attendance, attendance):
    for k in range(len(all)):
        if (all[j][k] == 1):
            all["attendance"][k] = i
        
# -------------------------------------------------------------------------
# 該当(最低出勤日数/週)のデータを削除
# -------------------------------------------------------------------------
all = all.drop(attendance, axis=1)

# 残業に関するデータについて

In [22]:
overtime = ["残業なし", "残業月10時間未満", "残業月20時間未満", "残業月20時間以上"]
number_overtime = [0, 10, 20, 30]

# ---------------------------------------------------------------------------------
# 残業時間 を求める．
# ---------------------------------------------------------------------------------
all["overtime"] = 0
for i, j in zip (number_overtime, overtime):
    for k in range(len(all)):
        if (all[j][k] == 1):
            all["overtime"][k] = i
        
# ---------------------------------------------------------------------------------
# 該当(残業時間)のデータを削除
# ---------------------------------------------------------------------------------
all = all.drop(overtime, axis=1)

# 勤務時間に関するデータについて

In [23]:
business_low_time = ["短時間勤務OK(1日4h以内)", "1日7時間以下勤務OK"]
number_business_low_time = ["4", "7"]

# -------------------------------------------------------------------
# 要求されている時間を求める．デフォルトは9時間とする．
# -------------------------------------------------------------------
all["business_low_time"] = 9
for i, j in zip (number_business_low_time, business_low_time):
    for k in range(len(all)):
        if (all[j][k] == 1):
            all["business_low_time"][k] = i

# -------------------------------------------------------------------
# 該当(残業時間)のデータを削除
# -------------------------------------------------------------------
all = all.drop(business_low_time, axis=1)

# 経験の有無について

In [25]:
experience = ["未経験OK", "経験者優遇"]

# ------------------------------------------------------------------
# 経験値の有無をまとめる
# ------------------------------------------------------------------
all["experience"] = 2
for i in range(len(all)):
    if ((all["未経験OK"][i] == 0) & (all["経験者優遇"][i] == 1)):
        all["experience"][i] = 1
        
    elif ((all["未経験OK"][i] == 1) & (all["経験者優遇"][i] == 0)):
        all["experience"][i] = 0

# ------------------------------------------------------------------
# 該当(経験)のデータを削除
# ------------------------------------------------------------------
all = all.drop(experience, axis=1)

# 英語力の有無について

In [26]:
english = ["英語力不要", "英語力を活かす"]
number_english = [0, 1, 2]

# -----------------------------------------
# 要求されている語学力を求める．
# -----------------------------------------
all["english"] = 1
for i, j in zip (number_english, english):
    for k in range(len(all)):
        if (all[j][k] == 1):
            all["english"][k] = i

# -----------------------------------------
# 該当(英語力)のデータを削除
# -----------------------------------------
all = all.drop(english, axis=1)

# Microsoftのソフトを何種類使えるか

In [27]:
microsoft = ["Wordのスキルを活かす", "Excelのスキルを活かす", "PowerPointのスキルを活かす"]

# -----------------------------------------------------------------------------------------
# 要求されているMicrosoftの扱える数を求める．
# -----------------------------------------------------------------------------------------
all["microsoft"] = 0
for i in microsoft:
    for j in range(len(all)):
        if (all[i][j] == 1):
            all["microsoft"][k] = all["microsoft"][k] + 1

# -----------------------------------------------------------------------------------------
# 該当(Microsoft関連のソフト)のデータを削除
# -----------------------------------------------------------------------------------------
all = all.drop(microsoft, axis=1)

# 服装について

In [28]:
wear = ["制服あり", "服装自由"]

# -------------------------------
# 服装自由なら1にする．
# -------------------------------
all["wear"] = 0
for i in range(len(all)):
    if (all["服装自由"][i] == 1):
        all["wear"][i] = 1

# -------------------------------
# 該当(服装)のデータを削除
# -------------------------------
all = all.drop(wear, axis=1)

In [29]:
# --------------------------------------------------------------------------------------------------------------------------------
# 変数の型がobjectであるものを抽出
# --------------------------------------------------------------------------------------------------------------------------------
object = ["掲載期間　開始日", "動画コメント", "休日休暇　備考", "（派遣）応募後の流れ", "期間・時間　勤務時間", "勤務地　備考", 
          "拠点番号", "お仕事名", "期間・時間　勤務開始日", "動画タイトル", "仕事内容", "勤務地　最寄駅1（沿線名）", "応募資格",
          "派遣会社のうれしい特典", "掲載期間　終了日", "お仕事のポイント（仕事PR）", "動画ファイル名", "勤務地　最寄駅1（駅名）"]

all[object]

,掲載期間 開始日,動画コメント,休日休暇 備考,（派遣）応募後の流れ,期間・時間 勤務時間,勤務地 備考,拠点番号,お仕事名,期間・時間 勤務開始日,動画タイトル,仕事内容,勤務地 最寄駅1（沿線名）,応募資格,派遣会社のうれしい特典,掲載期間 終了日,お仕事のポイント（仕事PR）,動画ファイル名,勤務地 最寄駅1（駅名）
0,2019/11/27,「未経験OK」「土日祝休み」「残業ほぼナシ」など、希望の条件にこだわって探せる♪*お仕事件数...,※土・日・祝がお休みです。※企業カレンダーあります。,＊最後までご覧いただき、ありがとうございます＊<BR><BR>「わたしでもチャレンジできるか...,9:30〜16:30 <BR>※残業はほとんどありません。<BR>※休憩は６０分です。,東京都足立区,206東京,ほぼ１６時半終業☆大学＊申請受付ｅｔｃ！未経験スタートＯＫ☆,2020/1/6,こだわりの条件でワガママに探せる＊,１月スタート！実働短め☆残業ほとんどなし×基本土日祝お休み！朝は少しゆっくりの９：３０出勤で...,東京メトロ千代田線,◆未経験者歓迎！<BR>【使用するＯＡスキル】Ｅｘｃｅｌ（ＶＬＯＯＫ関数）<BR><BR>▼...,"●当社は全国17,000件のお仕事を取り扱っており、毎日100名以上の方が就業を決めています...",2020/11/27,◆駅からスグ！飲食店・コンビニが近く便利！職場環境ＧＯＯＤ◎！当社含む派遣スタッフ多数活躍中...,022_190725.mp4,北千住
1,2019/11/27,「未経験OK」「土日祝休み」「残業ほぼナシ」など、希望の条件にこだわって探せる♪*お仕事件数...,※土・日・祝がお休みです。,＊最後までご覧いただき、ありがとうございます＊<BR><BR>「わたしでもチャレンジできるか...,9:00〜17:00 <BR>※残業はほとんどありません。<BR>※休憩は６０分です。,東京都千代田区,206東京,≪3ヶ月後正社員！≫大手損保会社でコツコツ事務＊社員食堂あり,2019/11/26,こだわりの条件でワガママに探せる＊,＜大手損保会社＞産休・育休取得実績ありの企業さんです♪ランチに嬉しい社員食堂完備の職場ですよ...,東京メトロ千代田線,【専門・高専卒以上】◆事務経験・簿記３級程度の知識がある方歓迎します。<BR>【使用するＯＡ...,"●当社は全国17,000件のお仕事を取り扱っており、毎日100名以上の方が就業を決めています...",2020/11/27,◆開始日相談可能です♪複数路線が利用でき便利！休憩室完備の自社ビル！当社スタッフさん就業中！...,022_190725.mp4,新御茶ノ水
2,2019/11/27,「未経験OK」「土日祝休み」「残業ほぼナシ」など、希望の条件にこだわって探せる♪*お仕事件数...,※土・日・祝がお休みです。,＊最後までご覧いただき、ありがとうございます＊<BR><BR>「わたしでもチャレンジできるか...,9:00〜17:00 <BR>※残業はほとんどありません。<BR>※休憩は６０分です。,東京都千代田区,206東京,≪3ヶ月後正社員！≫大手損保会社でコツコツ事務＊社員食堂あり,2019/11/26,こだわりの条件でワガママに探せる＊,＜大手損保会社＞産休・育休取得実績ありの企業さんです♪ランチに嬉しい社員食堂完備の職場ですよ...,東京メトロ千代田線,【専門・高専卒以上】◆事務経験・簿記３級程度の知識がある方歓迎します。<BR>【使用するＯＡ...,"●当社は全国17,000件のお仕事を取り扱っており、毎日100名以上の方が就業を決めています...",2020/11/27,◆開始日相談可能です♪複数路線が利用でき便利！休憩室完備の自社ビル！当社スタッフさん就業中！...,022_190725.mp4,新御茶ノ水
3,2019/11/27,「未経験OK」「土日祝休み」「残業ほぼナシ」など、希望の条件にこだわって探せる♪*お仕事件数...,※土・日・祝がお休みです。,＊最後までご覧いただき、ありがとうございます＊<BR><BR>「わたしでもチャレンジできるか...,9:00〜17:30 <BR>※休憩は６０分です。,東京都中央区,206東京,【未経験でも6ヶ月後社員】賞与年4.5ヶ月分＊非営利団体で事務,2019/11/26,こだわりの条件でワガママに探せる＊,安心安定の非営利団体！落ち着いた雰囲気の職場環境です！<BR>【ＯＡ事務】<BR>会議や委員...,東京メトロ日比谷線,【大卒以上】◆未経験者歓迎！<BR>【使用するＯＡスキル】Ｅｘｃｅｌ（関数）<BR><BR>...,"●当社は全国17,000件のお仕事を取り扱っており、毎日100名以上の方が就業を決めています...",2020/11/27,◆最寄駅から徒歩圏内！複数路線が利用可能！近くにコンビニ・カフェがあり便利！分煙されており快...,022_190725.mp4,八丁堀
4,2019/11/27,「未経験OK」「土日祝休み」「残業ほぼナシ」など、希望の条件にこだわって探せる♪*お仕事件数...,※土・日・祝がお休みです。,＊最後までご覧いただき、ありがとうございます＊<BR><BR>「わたしでもチャレンジできるか...,9:00〜17:30 <BR>※休憩は６０分です。,東京都中央区,206東京,【未経験でも6ヶ月後社員】賞与年4.5ヶ月分＊非営利団体で事務,2019/11/26,こだわりの条件でワガママに探せる＊,安心安定の非営利団体！落ち着いた雰囲気の職場環境です！<BR>【ＯＡ事務】<BR>会議や委員...,東京メトロ日比谷線,【大卒以上】◆未経験者歓迎！<BR>【使用するＯＡスキル】Ｅｘｃｅｌ（関数）<BR><BR>...,"●当社は全国17,000件のお仕事を取り扱っており、毎日100名以上の方が就業を決めています...",2020/11/27,◆最寄駅から徒歩圏内！複数路線が利用可能！近くにコンビニ・カフェがあり便利！分煙されており快...,022_190725.mp4,八丁堀
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19239,2019/10/24,「未経験OK」「土日祝休み」「残業ほぼナシ」など、希望の条件にこだわって探せる♪*お仕事件数...,※土・日・祝がお休みです。,＊最後までご覧いただき、ありがとうございます＊<BR><BR>「わたしでもチャレンジできるか...,9:00〜18:00 <BR>※休憩６０分。９時半〜１８時半／１０時〜１９時の勤務もあり。,東京都豊島区,206東京,基本定時まで！未経験もＯＫ！確認業務など！サンシャイン６０！,2019/10/23,こだわりの条件でワガママに探せる＊,［法律事務所］同業務に就業している方がいるので安心！キレイなオフイスで気持ちよくお仕事ができ...,東京メトロ有楽町線,◆未経験者歓迎！<BR>※タッチタイピングができる方＆コールセンター経験がある方歓迎。<BR...,"●当社は全国17,000件のお仕事を取り扱っており、毎日100名以上の方が就業を決めています...",2020/10/24,◆飲食店・コンビニが近くお昼はラクチン！当社スタッフ就業中で心強い！オフィカジＯＫ♪禁煙なの...,022_190725.mp4,東池袋
19240,2019/10/24,「未経験OK」「土日祝休み」「残業ほぼナシ」など、希望の条件にこだわって探せる♪*お仕事件数...,※土・日・祝がお休みです。,＊最後までご覧いただき、ありがとうございます＊<BR><BR>「わたしでもチャレンジできるか...,9:00〜17:25 <BR>※残業は月１０〜２０時間程度と少なめ。<BR>※休憩は４５分です。,東京都港区,206東京,◎引継ぎあり◎大手メーカー／保守契約管理業務＠1800円,2019/10/23,こだわりの条件でワガママに探せる＊,＜複合機・プリンターなどの精密機器メーカー＞残業少なめでプライベートとの両立も◎です！<BR...,山手線,◆業界経験問いません、ある方歓迎！<BR>※営業事務の経験が必要です。<BR>【使用するＯＡ...,"●当社は全国17,000件のお仕事を取り扱っており、毎日100名以上の方が就業を決めています...",2020/10/24,◆開始日相談可能です♪浜松町駅から徒歩圏内！近くには飲食店・コンビニもあり便利！オフィカジＯ...,022_190725.mp4,浜松町
19241,2019/10/24,「未経験OK」「土日祝休み」「残業ほぼナシ」など、希望の条件にこだわって探せる♪*お仕事件数...,※土・日・祝がお休みです。※週４日勤務もご相談可能です。,＊最後までご覧いただき、ありがとうございます＊<BR><BR>「わたしでもチャレンジできるか...,10:00〜18:00 <BR>※残業はほとんどありません。<BR>※休憩は６０分です。,東京都品川区,206東京,外資系／１８時まで☆未経験ＯＫ＠1700！データ入力など☆,2019/10/23,こだわりの条件でワガママに探せる＊,〈日用品卸売会社〉目黒駅からすぐ！カジュアル＆デニムでの就業ＯＫ！残業がほとんどないお仕事で...,山手線,◆業界経験問いません、ある方歓迎！<BR>※経理事務の経験が必要です。<BR>※仕訳業務がで...,"●

# "期間・時間　勤務時間" において，開始/終了 時間・勤務時間・休憩時間を抽出

In [30]:
import re
import mojimoji


all["launch_work_time"], all["end_work_time"], all["work_time"], all["rest_time"] = 0, 0, 0, 0

for i in range(len(all)):
    # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # 開始/終了 時間(日付が変わってから何時間後か)を計算
    # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    work_regex = re.compile((r'(\d+):(\d+)〜(\d+):(\d+)'))
    work_time = work_regex.search(all["期間・時間　勤務時間"][i]).group()
    
    all["launch_work_time"][i] = work_time.split('〜')[0]  # この段階では 9:00 のような形で格納されている．
    all["launch_work_time"][i] = (int(all["launch_work_time"][i].split(':')[0])*60 + int(all["launch_work_time"][i].split(':')[1]))/60
    
    all["end_work_time"][i] = work_time.split('〜')[1]
    all["end_work_time"][i] = (int(all["end_work_time"][i].split(':')[0])*60 + int(all["end_work_time"][i].split(':')[1]))/60
    
    all["launch_work_time"][i], all["end_work_time"][i] = round(all["launch_work_time"][i], 3), round(all["end_work_time"][i], 3)
    
    
    
    # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # 休憩時間の抽出，勤務時間の計算
    # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    all["期間・時間　勤務時間"][i] = mojimoji.zen_to_han(all["期間・時間　勤務時間"][i]) # 数字を半角変換
    
    # 休憩がない求人に着目し，労働時間を計算
    if (("休憩はありません" in all["期間・時間　勤務時間"][i]) | ("休憩はなし分です" in all["期間・時間　勤務時間"][i]) | ("休憩はなしです" in all["期間・時間　勤務時間"][i]) | ("休憩は有りません" in all["期間・時間　勤務時間"][i]) | ("休憩なし" in all["期間・時間　勤務時間"][i])):
        all["work_time"][i] = all["end_work_time"][i] - all["launch_work_time"][i]
        continue
        
    # 休憩がある求人に着目し，労働時間を計算
    if ("休憩" in all["期間・時間　勤務時間"][i]):
        all["期間・時間　勤務時間"][i] = all["期間・時間　勤務時間"][i].replace("休憩", "rest").replace("は", "").replace("計", "").replace("交替制で", "").replace("交代制で", "").replace("交替制", "").replace("交代制", "")
        rest_regex = re.compile((r'rest(\d+)'))
        rest_time = rest_regex.search(all["期間・時間　勤務時間"][i]).group().replace("rest", "")  # 何分か
        
        #all["rest_time"][i] = int(rest_time) / 60
        all["work_time"][i] = all["end_work_time"][i] - all["launch_work_time"][i] - round(int(rest_time) / 60, 3)
        all["rest_time"][i] = int(rest_time)
        
    # 休憩の記載がない求人に着目し，労働時間を計算
    else:
        all["work_time"][i] = all["end_work_time"][i] - all["launch_work_time"][i]
        
        
all["launch_work_time"], all["end_work_time"] = all["launch_work_time"].astype("float"), all["end_work_time"].astype("float")

# 拠点番号をカウントエンコーディング

In [31]:
# 各都道府県の名前と登場回数を定義する辞書
prefecture_figure = {}
# 拠点番号の種類をまとめてリスト化
region = all["拠点番号"].unique().tolist()

# ---------------------------------------------------------------------------
# 各拠点番号が何個あるのかを辞書として保存
# ---------------------------------------------------------------------------
for i in region:
    prefecture_figure[i] = all["拠点番号"].loc[all["拠点番号"] == i].count()
    

all["region"] = 0
# ---------------------------------------------------------------------------
# 各拠点番号に対して，辞書のkeyとセットになっているvalueを代入
# ---------------------------------------------------------------------------
for key, value in zip (prefecture_figure.keys(), prefecture_figure.values()):
    index = all.loc[all["拠点番号"] == key].index
    all["region"][index] = value

# 勤務地　最寄駅1（沿線名）をカウントエンコーディング

In [ ]:
# 各沿線名の名前と登場回数を定義する辞書
line_figure = {}
# 沿線名の種類をまとめてリスト化
line = all["勤務地　最寄駅1（沿線名）"].unique().tolist()

# 各沿線名が何個あるのかを辞書として保存
for i in line:
    line_figure[i] = all["勤務地　最寄駅1（沿線名）"].loc[all["勤務地　最寄駅1（沿線名）"] == i].count()
    
all["line"] = 0
# 各沿線名に対して，辞書のkeyとセットになっているvalueを代入    
for key, value in zip (line_figure.keys(), line_figure.values()):
    index = all.loc[all["勤務地　最寄駅1（沿線名）"] == key].index
    all["line"][index] = value

In [ ]:
all = all.drop(object, axis=1)

all_column = all.columns
all.dtypes

In [196]:
all

,職場の様子,勤務地固定,大手企業,交通費別途支給,職種コード,ルーティンワークがメイン,駅から徒歩5分以内,対象者設定 年齢下限,学校・公的機関（官公庁）,給与/交通費 給与支払区分,...,experience,english,microsoft,wear,launch_work_time,end_work_time,work_time,rest_time,region,line
0,3,1,0,0,22010,1,1,0,1,3,...,2,0,0,1,9.5,16.500,6,60,11527,276
1,3,1,1,0,20020,1,1,0,0,3,...,1,0,0,1,9.0,17.000,7,60,11527,276
2,3,1,1,0,20020,1,1,0,0,3,...,1,0,0,1,9.0,17.000,7,60,11527,276
3,2,1,0,0,20020,1,0,0,1,3,...,2,0,0,1,9.0,17.500,7,60,11527,496
4,2,1,0,0,20020,1,0,0,1,3,...,2,0,0,1,9.0,17.500,7,60,11527,496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19239,3,1,0,0,20020,1,0,0,0,3,...,2,0,0,1,9.0,18.000,8,60,11527,482
19240,3,1,1,0,20030,1,0,0,0,3,...,1,0,0,1,9.0,17.417,7,45,11527,4037
19241,3,1,0,0,22020,1,1,0,0,3,...,2,0,0,1,10.0,18.000,7,60,11527,4037
19242,3,1,1,0,20030,1,0,0,0,3,...,2,0,0,0,9.0,17.500,7,45,11527,371


# データを標準化，trainとtestに分ける

In [248]:
scaler = preprocessing.StandardScaler()
all = scaler.fit_transform(all)
scalar_all = pd.DataFrame(all,  columns=all_column)

train = scalar_all[:15853]
test = scalar_all[15853:].reset_index(drop=True)

# 重回帰モデルの作成

In [249]:
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# 交差検証の結果を記録するリスト
scores = []
# ---------------------------------------------------------
# 予測モデルの作成
# ---------------------------------------------------------
clf = linear_model.LinearRegression()


# ---------------------------------------------------------
# 交差検証の実施
# ---------------------------------------------------------
kf = KFold(n_splits=5, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    tr_x, va_x = train.iloc[tr_idx], train.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        
    model = clf.fit(tr_x, tr_y)
    va_pred = model.predict(va_x)
    
    va_pred = np.where(va_pred <= 0, 0, va_pred)
    
    score = mean_squared_error(va_y, va_pred)
    print(score)
    scores.append(score)

print(np.sqrt(np.mean(scores)))


predict = model.predict(test)
predict = np.where(predict <= 0, 0, predict)
# bestRMSE: 0.7012787167492093

0.7012787167492093


In [178]:
predict

array([1.04333203, 1.42738368, 1.6029385 , ..., 0.84195354, 0.        ,
       0.32381084])

# [予測]解答の作成

In [107]:
base = np.zeros((3391, 2))

ans = pd.DataFrame(base, columns=["お仕事No.", "応募数 合計"])
ans["お仕事No."] = test_No
ans["応募数 合計"] = predict

ans.to_csv("answer.csv", index=False)